# Про градиентный спуск

Сравнить сложность точного решения задачи линейной регрессии для случая квадратичной ошибки и сложность решения этой же задачи методом градиентного спуска (при условии, что градиентный спуск сходится с K шагов, в задаче есть M признаков и длина обучающей выборки равна N. N>K и N>M)

Пусть:<br>
$X$ - матрица объекты-признаки (обучающая выборка), $X \in \mathbb{R}^{N \times M}$<br>
$y$ - вектор целевой переменной, $y \in \mathbb{R}^N$<br>
Вычисление предсказания: $\hat{y} = Xw$<br>
Лосс: $L = \frac{1}{N} || Xw - y ||^2_2$<br>
Градиент по весам: $\nabla _w{L} = \frac{2}{N} X^T (Xw - y)$

## Сложность точного решения

Для решения задачи $L \to \displaystyle\min_{w}$ приравняем градиент к 0:<br>
$\frac{2}{N} X^T (Xw - y) = 0 \\
(X^T X)w = X^T y \\
w = (X^T X)^{-1} X^T y
$

Рассчитаем сложность точного решения.<br>
Перемножение матриц $X^T X$ за $O(M^2 N)$<br>
Обращение матрицы $(X^T X)^{-1}$ за $O(M^3)$<br>
Перемножение матрицы $X^T$ на вектор $y$ за $O(MN)$<br>
Перемножение матрицы $(X^T X)^{-1}$ на вектор $X^T y$ за $O(M^2)$<br>
Итоговая сложность $O(M^2 N + M^3 + MN + M^2) \sim O(M^2 N + M^3)$

## Сложность методом градиентного спуска

$w_{i+1} = w_i - \eta \nabla _w L$

Рассчитаем сложность 1 шага градиентного спуска.<br>
Вычисление предсказания $\hat{y} = Xw$ за $O(NM)$<br>
Вычисление градиента $\nabla _w L = \frac{2}{N} X^T (\hat{y} - y)$ за $O(MN)$<br>
Обновление весов $w_{i+1} = w_i - \eta \nabla _w L$ за $O(M)$<br>
Сложность 1 шага $O(NM + MN + M) \sim O(MN)$

Если градиентный спуск сходится за $K$ шагов, то итоговая сложность равна $O(MNK)$.

## Вывод

Таким образом, после сравнения сложности точного решения и сложности градиентного спуска, видим, что применение метода градиентного спуска позволяет избавиться от требовательной операции обращения матрицы за $O(M^3)$.

# Про FWL-Теорему

Кажется, я успешно успел вам наврать на лекции про "отпиливание" зависимости из Y-переменной (я вроде это забыл сказать), поэтому реабилитироваться буду примером:

In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
 
np.random.seed(42069)
 
# Пусть у нас есть набор данных, где есть линейная зависимость y от X1 и X2
# При этом X1 и X2 тоже малясь зависимые
df = pd.DataFrame({'x1': np.random.uniform(0, 10, size=1000)})
df['x2'] = 4.9 + df['x1'] * 0.983 + 2.104 * np.random.normal(0, 1.35, size=1000)
df['y'] = 8.643 - 2.34 * df['x1'] + 3.35 * df['x2'] + np.random.normal(0, 1.65, size=1000)
df['const'] = 1
 
# Построим линейную регрессию-МНК из 1, X1 и X2
model = sm.OLS(
    endog=df['y'],
    exog=df[['const', 'x1', 'x2']]
).fit()

# Внимательно смотрим на коэффициент при X2
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.972
Model:                            OLS   Adj. R-squared:                  0.972
Method:                 Least Squares   F-statistic:                 1.754e+04
Date:                Mon, 14 Oct 2024   Prob (F-statistic):               0.00
Time:                        15:45:08   Log-Likelihood:                -1934.3
No. Observations:                1000   AIC:                             3875.
Df Residuals:                     997   BIC:                             3889.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.6842      0.139     62.606      0.000       8.412       8.956
x1            -2.3455      0.027    -88.274      0.000      -2.398      -2.293
x2             3.3544      0.019    178.202      0.000       3.317       3.391
==============================================================================
Omnibus:                        0.497   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.780   Jarque-Bera (JB):                0.394
Skew:                          -0.036   Prob(JB):                        0.821
Kurtosis:                       3.066   Cond. No.                         31.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
# Научим регрессию X1 на X2
model_x2 = sm.OLS(
    endog=df['x2'],
    exog=df[['const', 'x1']]
).fit()

# Научим регрессию X1 на y
model_yx1 = sm.OLS(
    endog=df['y'],
    exog=df[['const', 'x1']]
).fit()

In [8]:
# Полученными регрессиями "предскажем X2 и Y"

df['yx1'] = model_yx1.predict(df[['const', 'x1']])
df['x2x1'] = model_x2.predict(df[['const', 'x1']])

In [9]:
# А затем "отпилим" предсказание из данных
df['y_detrended'] = df['y'] - df['yx1']
df['x2_detrended'] = df['x2'] - df['x2x1']

In [11]:
# Учим модель на "очищенных" переменных и, о Боже, коэффициент при X2 остается "как был"
model_detrended = sm.OLS(
    endog=df['y_detrended'],
    exog=df[['const', 'x2_detrended']]
).fit()
model_detrended.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            y_detrended   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                 3.179e+04
Date:                Mon, 14 Oct 2024   Prob (F-statistic):               0.00
Time:                        16:14:48   Log-Likelihood:                -1934.3
No. Observations:                1000   AIC:                             3873.
Df Residuals:                     998   BIC:                             3882.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.002e-14      0.053  -1.89e-13      1.000      -0.104       0.104
x2_detrended     3.3544      0.019    178.292      0.000       3.318       3.391
==============================================================================
Omnibus:                        0.497   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.780   Jarque-Bera (JB):                0.394
Skew:                          -0.036   Prob(JB):                        0.821
Kurtosis:                       3.066   Cond. No.                         2.82
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# А теперь задачка :)

Возьмите данные c kaggle, например [отсюда](https://www.kaggle.com/code/malakalaabiad/house-prices-techniques/input) и удостоверьтесь, что FWL-теорема работает, но только не для случая одной переменной :)


In [124]:
# Загружаем датасет, смотрим значения признаков - все числовые
data = pd.read_csv('boston.csv')
data['CONST'] = 1

print('Размерность:', data.shape)
print('Названия признаков:', data.columns.values)
data.head()

Размерность: (506, 15)
Названия признаков: ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT' 'MEDV' 'CONST']


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,CONST
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0,1
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6,1
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7,1
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4,1
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2,1


In [137]:
# Названия признаков и целевая переменная
feature_columns = data.columns.drop(['MEDV', 'CONST'])
const_column = 'CONST'
target_column = 'MEDV'

In [138]:
# Строим линейную регрессию-МНК из всех признаков
model = sm.OLS(
    endog=data[target_column],
    exog=data[feature_columns.to_list() + [const_column]]
).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Mon, 14 Oct 2024   Prob (F-statistic):          6.72e-135
Time:                        18:32:08   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.1080      0.033     -3.287      0.001      -0.173      -0.043
ZN             0.0464      0.014      3.382      0.001       0.019       0.073
INDUS          0.0206      0.061      0.334      0.738      -0.100       0.141
CHAS           2.6867      0.862      3.118      0.002       0.994       4.380
NOX          -17.7666      3.820     -4.651      0.000     -25.272     -10.262
RM             3.8099      0.418      9.116      0.000       2.989       4.631
AGE            0.0007      0.013      0.052      0.958      -0.025       0.027
DIS           -1.4756      0.199     -7.398      0.000      -1.867      -1.084
RAD            0.3060      0.066      4.613      0.000       0.176       0.436
TAX           -0.0123      0.004     -3.280      0.001      -0.020      -0.005
PTRATIO       -0.9527      0.131     -7.283      0.000      -1.210      -0.696
B              0.0093      0.003      3.467      0.001       0.004       0.015
LSTAT         -0.5248      0.051    -10.347      0.000      -0.624      -0.425
CONST         36.4595      5.103      7.144      0.000      26.432      46.487
==============================================================================
Omnibus:                      178.041   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              783.126
Skew:                           1.521   Prob(JB):                    8.84e-171
Kurtosis:                       8.281   Cond. No.                     1.51e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.51e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [140]:
# Разделяем признаки на две группы
x1_features = np.random.choice(feature_columns, np.random.randint(2, 10), replace=False)
x2_features = feature_columns.drop(x1_features).to_numpy()
print(x1_features)
print(x2_features)

['RAD' 'RM' 'B' 'TAX' 'INDUS' 'NOX' 'AGE' 'DIS']
['CRIM' 'ZN' 'CHAS' 'PTRATIO' 'LSTAT']


In [145]:
# Учимся предсказывать признаки группы X2 с помощью признаков из группы X1
for feature in x2_features:
    model_feature_by_x1 = sm.OLS(
        endog=data[feature],
        exog=data[x1_features.tolist() + [const_column]]
    ).fit()

    # Отнимаем предсказанные значения признака
    data[feature + '_by_x1'] = model_feature_by_x1.predict(data[x1_features.tolist() + [const_column]])
    data[feature + '_detrended'] = data[feature] - data[feature + '_by_x1']

# Учимся предсказывать целевую переменную с помощью признаков из X1
model_MEDV = sm.OLS(
    endog=data[target_column],
    exog=data[x1_features.tolist() + [const_column]]
).fit()

# Отнимаем предсказанные значения таргета
data[target_column + '_by_x1'] = model_MEDV.predict(data[x1_features.tolist() + [const_column]])
data[target_column + '_detrended'] = data[target_column] - data[target_column + '_by_x1']

In [146]:
# Обучаем регрессию скорректированного таргета на скорректированных признаках группы X2
model_detrended = sm.OLS(
    endog=data[target_column + '_detrended'],
    exog=data[list(map(lambda name: name + '_detrended', x2_features)) + [const_column]]
).fit()

In [150]:
# Проверяем коэффициенты при признаках группы X2

print(f'Коэффициенты в линейной регрессии, обученной на всех признаках:\n{model.params[x2_features]}\n')

print(f'Коэффициенты в линейной регрессии, обученной на скорректированных признаках из группы X2:\n{model_detrended.params}\n')

Коэффициенты в линейной регрессии, обученной на всех признаках:
CRIM      -0.108011
ZN         0.046420
CHAS       2.686734
PTRATIO   -0.952747
LSTAT     -0.524758
dtype: float64

Коэффициенты в линейной регрессии, обученной на скорректированных признаках из группы X2:
CRIM_detrended      -1.080114e-01
ZN_detrended         4.642046e-02
CHAS_detrended       2.686734e+00
PTRATIO_detrended   -9.527472e-01
LSTAT_detrended     -5.247584e-01
CONST               -8.740508e-13
dtype: float64



## Вывод

Видим, что все коэффициентики остались такими же.

# Про эквивалентность или не эквивалентность разных методов подсчета квантилей

Сгенерируйте 2 выборки длины ,например, 10000 из:

1. Нормального
2. Логнормального
3. Экспоненциального

Распределений с наперед заданными параметрами, так чтобы вы могли однозначно посчитать разницу медиан (используя теорвер и википедию)



Проверьте, какой по этим выборкам будет получаться 95% доверительный интервал на разницу медиан, если его посчитать с помощью:

1. Бутстрепа
2. [Подгонки](https://engineering.atspotify.com/2022/03/comparing-quantiles-at-scale-in-online-a-b-testing/) от Spotify
3. [Подгонки](https://www.evanmiller.org/bootstrapping-sample-medians.html) результатов бутстрепа от Эвана Миллера
4. [Метода Прайса-Боннетта](https://www.tandfonline.com/doi/abs/10.1080/00949650212140)

Что вы можете сказать о работоспособности методов?

(можно попробовать подать на вход какие-то другие распределения, как бы провести "стресс-тест" метода)


In [182]:
from scipy import stats
from tqdm.auto import tqdm

## Бутстреп

In [248]:
def bootstrap_quantile(sample_1, sample_2, quantile=0.5, n_bootstrap_samples=10000, alpha=0.05):
    bootstrap_statistics = []
    quantile_diff = np.quantile(sample_1, quantile) - np.quantile(sample_2, quantile)

    for _ in tqdm(range(n_bootstrap_samples)):
        bootstrap_sample = np.random.choice(sample_1, size=len(sample_1), replace=True)
        bootstrap_median_1 = np.quantile(bootstrap_sample, quantile)

        bootstrap_sample = np.random.choice(sample_2, size=len(sample_2), replace=True)
        bootstrap_median_2 = np.quantile(bootstrap_sample, quantile)

        bootstrap_statistics.append(bootstrap_median_1 - bootstrap_median_2)


    z = stats.norm.ppf(1 - alpha/2)
    se = np.std(bootstrap_statistics)
    ci = (quantile_diff - z*se, quantile_diff + z*se)

    return quantile_diff, ci

### Нормальное

Для $N(\mu, \sigma^2)$ значение медианы равно $\mu$, т.е. совпадает со средним.

In [250]:
mu_1, sigma_1 = 2, 1.5
mu_2, sigma_2 = 0, 1

sample_1 = np.random.normal(mu_1, sigma_1, size=10000)
sample_2 = np.random.normal(mu_2, sigma_2, size=10000)

print('Теоретическая разница медиан:', mu_1 - mu_2)

Теоретическая разница медиан: 2


In [251]:
median_diff, ci = bootstrap_quantile(sample_1, sample_2)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [252]:
print('Разница медиан:', median_diff)
print('95% доверительный интервал:', ci)

Разница медиан: 2.024115018335167
95% доверительный интервал: (1.9790992451889673, 2.0691307914813666)


### Логнормальное

Для $LogN(\mu, \sigma^2)$ значение медианы равно $e^{\mu}$.

In [253]:
mu_1, sigma_1 = 1, 0.5
mu_2, sigma_2 = 2, 1.1

sample_1 = np.random.lognormal(mu_1, sigma_1, size=10000)
sample_2 = np.random.lognormal(mu_2, sigma_2, size=10000)

print('Теоретическая разница медиан:', np.exp(mu_1) - np.exp(mu_2))

Теоретическая разница медиан: -4.670774270471606


In [254]:
median_diff, ci = bootstrap_quantile(sample_1, sample_2)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [255]:
print('Разница медиан:', median_diff)
print('95% доверительный интервал:', ci)

Разница медиан: -4.704181914169902
95% доверительный интервал: (-4.916908051649697, -4.491455776690106)


### Экспоненциальное

Для $Exp(\lambda)$ значение медианы равно $\frac{ln(2)}{\lambda}$.

In [256]:
lambda_1 = 1
lambda_2 = 2 * np.log(2)

# в numpy по-дурацки задается параметр распределения, scale = 1/lambda
sample_1 = np.random.exponential(1/lambda_1, size=10000)
sample_2 = np.random.exponential(1/lambda_2, size=10000)

print('Теоретическая разница медиан:', np.log(2) / lambda_1 - np.log(2) / lambda_2)

Теоретическая разница медиан: 0.1931471805599453


In [257]:
median_diff, ci = bootstrap_quantile(sample_1, sample_2)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [258]:
print('Разница медиан:', median_diff)
print('95% доверительный интервал:', ci)

Разница медиан: 0.21406292933494192
95% доверительный интервал: (0.18946535024881664, 0.2386605084210672)


## Подгонка от Spotify

In [259]:
def spotify_approach(sample_1, sample_2, quantile=0.5, n_bootstrap_samples=5000, alpha=0.05):
    n = len(sample_1)

    median_diff = np.quantile(sample_1, quantile) - np.quantile(sample_2, quantile)

    # сортируем обе выборки
    sample_1_sorted = np.sort(sample_1)
    sample_2_sorted = np.sort(sample_2)

    # семплируем индексы из биномиального распределения
    sample_1_indices = np.random.binomial(n=n, p=quantile, size=n_bootstrap_samples) 
    sample_2_indices = np.random.binomial(n=n, p=quantile, size=n_bootstrap_samples)

    # вытаскиваем из отсортированной выборки элементы по индексам
    bootstrap_diff_distr = sample_1_sorted[sample_1_indices] - sample_2_sorted[sample_2_indices]

    # доверительный интервал
    ci = np.quantile(bootstrap_diff_distr, [alpha/2, 1 - alpha/2])

    return median_diff, ci

### Нормальное

In [260]:
mu_1, sigma_1 = 2, 1.5
mu_2, sigma_2 = 0, 1

sample_1 = np.random.normal(mu_1, sigma_1, size=10000)
sample_2 = np.random.normal(mu_2, sigma_2, size=10000)

print('Теоретическая разница медиан:', mu_1 - mu_2)

Теоретическая разница медиан: 2


In [261]:
median_diff, ci = spotify_approach(sample_1, sample_2)
print('Разница медиан:', median_diff)
print('95% доверительный интервал:', ci)

Разница медиан: 1.9714384026088305
95% доверительный интервал: [1.92344413 2.01102283]


### Логнормальное

In [262]:
mu_1, sigma_1 = 1, 0.5
mu_2, sigma_2 = 2, 1.1

sample_1 = np.random.lognormal(mu_1, sigma_1, size=10000)
sample_2 = np.random.lognormal(mu_2, sigma_2, size=10000)

print('Теоретическая разница медиан:', np.exp(mu_1) - np.exp(mu_2))

Теоретическая разница медиан: -4.670774270471606


In [263]:
median_diff, ci = spotify_approach(sample_1, sample_2)
print('Разница медиан:', median_diff)
print('95% доверительный интервал:', ci)

Разница медиан: -4.806570427010124
95% доверительный интервал: [-4.99981086 -4.57392346]


### Экспоненциальное

In [264]:
lambda_1 = 1
lambda_2 = 2 * np.log(2)

# в numpy по-дурацки задается параметр распределения, scale = 1/lambda
sample_1 = np.random.exponential(1/lambda_1, size=10000)
sample_2 = np.random.exponential(1/lambda_2, size=10000)

print('Теоретическая разница медиан:', np.log(2) / lambda_1 - np.log(2) / lambda_2)

Теоретическая разница медиан: 0.1931471805599453


In [265]:
median_diff, ci = spotify_approach(sample_1, sample_2)
print('Разница медиан:', median_diff)
print('95% доверительный интервал:', ci)

Разница медиан: 0.20350459612607597
95% доверительный интервал: [0.17916314 0.22875469]


## Пунктов 3 и 4 нет

## Вывод:

Подгонка от Spotify работает быстрее обычного бутстрепа. Дает хороший доверительный интервал.